<a href="https://colab.research.google.com/github/blackrosedragon2/Text-Summarization/blob/master/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tqdm
!pip install torchsummaryx
!pip install nltk
!pip install pyspellchecker


In [0]:
#Reading data onto zipfile object ZF
from google.colab import drive
import os 
import zipfile
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/data/'  
zf=zipfile.ZipFile(root_path+"cnn_stories.zip")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import re
from spellchecker import SpellChecker
import nltk
#nltk.download()
class preprocess:
    #Accepts a string and performs string operations on it
    def seperate(self,para):#1
        # find @highlight, add length and split story and summary, then decode to utf-8 format, use split tokenizer and seperate the 4 lines , split the \n parts and return the list
        story=para[:para.find('@highlight')]
        summary=[i.splitlines()[2] for i in (para[para.find('@highlight')+len('@highlight'):]).split('@highlight')]
        return story,summary    
        #returns (string,list)
    
    def spell_check(self,para):#2
        #corrects spellings(may not be used for us) + takes much longer to correct spellings (5+ minutes)
        spell = SpellChecker()
        para=para.split(' ')
        correctpara=[]
        for word in para:
            correctpara.append(spell.correction(word))
        return ' '.join(correctpara)

    def remove_apostrophes(self,para):#3
        return re.sub("(?<=[a-z])'(?=[a-z])", "", para)

    def remove_parentheses(self,para):#4
        #removes anything inside parentheses
        return re.sub(r'\([^)]*\)', '', para)

    def remove_sp(self,para):#5
        #removes special symbols
        return re.sub('[^A-Za-z0-9]+', ' ', para)   #note: add exception for words like U.N. 

    def remove_short(self,para):#6
        #removes words that are <3 in length
        para=para.split(' ')
        longpara=[]
        for word in para:
            if len(word) >= 3:
                longpara.append(word)
        return ' '.join(longpara)

    def add_tags(self,para):#7
        #add start and end tags 
        para="<start>"+para+"<end>"
        return para

    def tokenize(self,para):#8
        tokenlist = nltk.word_tokenize(para)
        return tokenlist

    def apply_all(self,para):#9
        #applies 1,3,4,5,6 functions, may need reordering 
        p=preprocess()

        story,summary=p.seperate(para.decode("utf-8"))

        #story=p.spell_check(story)
        #summary=p.spell_check(summary)

        story=p.remove_apostrophes(story)
        summary=[p.remove_apostrophes(i) for i in summary] 

        story=p.remove_parentheses(story)
        summary=[p.remove_parentheses(i) for i in summary]

        story=p.remove_sp(story)
        summary=[p.remove_sp(i) for i in summary]

        story=p.remove_short(story)# may not be needed
        summary=[p.remove_short(i) for i in summary]

        story=p.tokenize(story)
        summary=[p.tokenize(i) for i in summary]

        return story,summary



In [0]:
#Read content of all files onto x (takes approximatly 13 minutes for both modes)
from tqdm import tqdm

def get_data(preproc=0):
    i=1
    if preproc==1:
        stories=[]
        story=[]
        summaries=[]
        summary=[]
        p=preprocess()
        with tqdm(total=92600) as pbar:
            while(True):
                #Reads zip file until eof exception occurs
                i=i+1
                try:
                    story,summary=p.apply_all(zf.open(zf.namelist()[i]).read())
                    stories.append(story)
                    summaries.append(summary)
                except Exception as E:
                    break
                pbar.update(1)
        return stories,summaries
    else:
        x=[]
        with tqdm(total=92600) as pbar:
            while(True):
                #Reads zip file until eof exception occurs
                i=i+1
                try:
                    x.append(zf.open(zf.namelist()[i]).read())
                except Exception as E:
                    break
                pbar.update(1)
        return x

story,summary=get_data(1) 
for j,i in enumerate(summary):
    print(i)
    if j>10:
        break


100%|█████████▉| 92579/92600 [22:12<00:00, 69.49it/s]


[['Syrian', 'official', 'Obama', 'climbed', 'the', 'top', 'the', 'tree', 'doesnt', 'know', 'how', 'get', 'down'], ['Obama', 'sends', 'letter', 'the', 'heads', 'the', 'House', 'and', 'Senate'], ['Obama', 'seek', 'congressional', 'approval', 'military', 'action', 'against', 'Syria'], ['Aim', 'determine', 'whether', 'were', 'used', 'not', 'whom', 'says', 'spokesman']]
[['Usain', 'Bolt', 'wins', 'third', 'gold', 'world', 'championship'], ['Anchors', 'Jamaica', '4x100m', 'relay', 'victory'], ['Eighth', 'gold', 'the', 'championships', 'for', 'Bolt'], ['Jamaica', 'double', 'womens', '4x100m', 'relay']]
[['The', 'employee', 'agencys', 'Kansas', 'City', 'office', 'among', 'hundreds', 'virtual', 'workers'], ['The', 'employees', 'travel', 'and', 'from', 'the', 'mainland', 'last', 'year', 'cost', 'more', 'than', '000'], ['The', 'telecommuting', 'program', 'like', 'all', 'GSA', 'practices', 'under', 'review']]
[['NEW', 'Canadian', 'doctor', 'says', 'she', 'was', 'part', 'team', 'examining', 'Harry'

![alt text](https://pytorch.org/tutorials/_images/encoder-network.png)

In [0]:
%xmode Plain
import torch.nn as nn
import torch
import numpy as np
from torchsummaryX import summary

device = torch.device("cuda")

class GRU_Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, drop=0.2):
        super(GRU_Encoder, self).__init__()
        #GRU with dropout,fc layer, relu layer
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True, dropout=drop, bidirectional =True) 
        #batch first will create dims such that dimensions=(batch,dim1,dim2...)

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data      
         #grabs any parameter of the model and uses it to instantiate (through .data.new) a new tensor on the same device
        hidden = weight.new(2, batch_size, self.hidden_size).zero_().to(device)
        #ints a hidden vector shape( n_layer,batch_size,hidden_size) to zero , nlayer= 2 because bidirectional 
        return hidden

    def forward(self, x, h=0):# x dims=()
        if h==0:
            h=self.init_hidden(16)

        embedded = self.embedding(x).view(x.shape[0],-1,self.hidden_size)
        out=embedded
        out, h = self.gru(out, h)  
        print("X {},OUTPUT {},HIDDEN {}".format(x.shape,out.shape,h.shape))

        return out, h  #(output of encoder(give to-> attention mechanism ), next hidden state)

#input trensor dims must be (i/p dim,timesteps,batchsize)
gruEncoder_model=GRU_Encoder(input_size=10, hidden_size=128, output_size=10)
inputs = torch.zeros((16,10, 10), dtype=torch.long) # (batch_size, outputdim, inputdim)
inputs=inputs.cuda()
gruEncoder_model.cuda()
summary(gruEncoder_model,inputs)



Exception reporting mode: Plain
X torch.Size([16, 10, 10]),OUTPUT torch.Size([16, 100, 256]),HIDDEN torch.Size([2, 16, 128])
            Kernel Shape       Output Shape  Params  Mult-Adds
Layer                                                         
0_embedding    [128, 10]  [16, 10, 10, 128]    1280       1280
1_gru                  -     [16, 100, 256]  198144     196608
--------------------------------------------------------------
                      Totals
Total params          199424
Trainable params      199424
Non-trainable params       0
Mult-Adds             197888


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding,"[128, 10]","[16, 10, 10, 128]",1280,1280
1_gru,-,"[16, 100, 256]",198144,196608


![alt text](https://pytorch.org/tutorials/_images/attention-decoder-network.png)

In [0]:
import torch.nn.functional as F

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=15):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, encoder_outputs,hidden=1):
        if hidden==1:
            hidden=self.initHidden()
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1) #calculate a set of attention weight
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0)) #bmm atn_weight, encoder_op

        output = torch.cat((embedded[0], attn_applied[0]), 1) #concat embedded i/p vector and attn applied
        output = self.attn_combine(output).unsqueeze(0) #Apply linear layer to concat vector

        output = F.relu(output) #apply relu
        output, hidden = self.gru(output, hidden) #apply gru to hidden and transformed attention vector

        output = F.log_softmax(self.out(output[0]), dim=1)#softmax for output
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
